In [13]:
# Import the dependencies
import pandas as pd
import os
import gmaps
import gmaps.datasets
import requests
from config import g_key

In [14]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [15]:
#load files
weather_py_data_to_load = os.path.join("data","WeatherPy_vacation.csv")
weather_py_df = pd.read_csv(weather_py_data_to_load)
weather_py_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,17,Praia,CV,84.20,scattered clouds,14.92,-23.51,LT Aparthotel
1,19,Jetpur,IN,92.30,overcast clouds,21.73,70.62,Hotel The Grand Bansari
2,20,Kahului,US,78.80,overcast clouds,20.89,-156.47,Maui Seaside Hotel
3,34,Eskisehir,TR,80.60,few clouds,39.78,30.52,Hotel ibis Eskisehir
4,38,Faanui,PF,76.78,scattered clouds,-16.48,-151.75,Bora-Bora Pearl Beach Resort & Spa
...,...,...,...,...,...,...,...,...
170,552,Trairi,BR,82.26,scattered clouds,-3.28,-39.27,Pousada Field Lilies - Trairi
171,553,Bambanglipuro,ID,76.08,clear sky,-7.95,110.28,Puri Brata
172,558,Toul,FR,80.60,scattered clouds,48.67,5.92,Abc Hotel
173,563,Bosanska Gradiska,BA,89.60,few clouds,45.14,17.25,Hostel Art Gradiska


In [16]:
# Using the template add the hotel marks to the map
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Hotel Name</dt><dd>{Hotel Name} at {Max Temp}</dd>
"""

# Store the DataFrame row
hotel_info = [info_box_template.format(**row) for index, row in weather_py_df.iterrows()]

# Add a marker layer map of the vacation spots and a pop-up marker for each city
locations = weather_py_df[["Lat", "Lng"]]
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(38,38), zoom_level = 1.8)
fig.add_layer(marker_layer)
fig


Figure(layout=FigureLayout(height='420px'))

In [17]:
print("Please list 4 cities that are close to each other on the map")

Please list 4 cities that are close to each other on the map


In [18]:
#prompt for city
city1 = input("Enter first city ")
city2 = input("Enter second city ")
city3 = input("Enter third city ")
city4 = input("Enter fourth city ")

Enter first city Puerto Escondido
Enter second city Lazaro Cardenas
Enter third city San Patricio
Enter fourth city Ixtapa


In [19]:
#loc for separate databases
city1 = weather_py_df.loc[weather_py_df["City"] == city1]
city2 = weather_py_df.loc[weather_py_df["City"] == city2]
city3 = weather_py_df.loc[weather_py_df["City"] == city3]
city4 = weather_py_df.loc[weather_py_df["City"] == city4]

In [20]:
print(city1)
print(city2)
print(city3)
print(city4)

     City_ID              City Country  Max Temp Current Description    Lat  \
174      564  Puerto Escondido      MX      77.0       broken clouds  15.85   

       Lng                     Hotel Name  
174 -97.07  Aldea del Bazar Hotel and Spa  
     City_ID             City Country  Max Temp Current Description    Lat  \
168      548  Lazaro Cardenas      MX     77.74     overcast clouds  17.96   

       Lng            Hotel Name  
168 -102.2  Hotel Quinta Antigua  
    City_ID          City Country  Max Temp Current Description    Lat    Lng  \
33      135  San Patricio      MX     76.77       broken clouds  19.22 -104.7   

           Hotel Name  
33  Hotel Cabo Blanco  
     City_ID    City Country  Max Temp Current Description   Lat    Lng  \
107      355  Ixtapa      MX      77.0    scattered clouds  20.7 -105.2   

                        Hotel Name  
107  Container Inn Puerto Vallarta  


In [21]:
#make tuples of all the cities
city1_co = city1["Lat"].values[0], city1["Lng"].values[0]
city2_co = city2["Lat"].values[0], city2["Lng"].values[0]
city3_co = city3["Lat"].values[0], city3["Lng"].values[0]
city4_co = city4["Lat"].values[0], city4["Lng"].values[0]
print(city1_co)
print(city2_co)
print(city3_co)
print(city4_co)

(15.85, -97.07)
(17.96, -102.2)
(19.22, -104.7)
(20.7, -105.2)


In [22]:
#route between the 4 cities. city 1=A, used waypoints for all the stops
fig = gmaps.figure()
city1_co2city4_co = gmaps.directions_layer(city1_co, city4_co, waypoints =[city2_co,city3_co],travel_mode="DRIVING")
fig.add_layer(city1_co2city4_co)
fig

Figure(layout=FigureLayout(height='420px'))

In [23]:
#merge the city dataframes
city_merge_df = pd.concat([city1, city2, city3, city4])
city_merge_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
174,564,Puerto Escondido,MX,77.00,broken clouds,15.85,-97.07,Aldea del Bazar Hotel and Spa
168,548,Lazaro Cardenas,MX,77.74,overcast clouds,17.96,-102.20,Hotel Quinta Antigua
33,135,San Patricio,MX,76.77,broken clouds,19.22,-104.70,Hotel Cabo Blanco
107,355,Ixtapa,MX,77.00,scattered clouds,20.70,-105.20,Container Inn Puerto Vallarta


In [24]:
#popup marker for 4 cities
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Hotel Name</dt><dd>{Hotel Name} at {Max Temp}</dd>
"""

# Store the DataFrame row
hotel_info = [info_box_template.format(**row) for index, row in city_merge_df.iterrows()]

locations = city_merge_df[["Lat", "Lng"]]
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
fig


Figure(layout=FigureLayout(height='420px'))